1) Dado los acontecimientos en USA, deseamos obtener datos que
nos den mayor información sobre las muertes de gente de raza negra
por parte de oficiales de policía.
Para ello, tenemos un csv con información sobre las muertes por
parte de oficiales de policía en USA para 2015 hasta 2017:

(name, date, race, city, state)

Y otro csv con información sobre el porcentaje de pobreza en las
ciudades de USA:

(state, city, poverty_rate)

Se pide:

a) Obtener las 10 ciudades con mayor diferencia entre el porcentaje
de pobreza de la ciudad y el porcentaje de pobreza del estado en el
que se encuentra esa ciudad. Por ejemplo si en la ciudad de Houston
la pobreza es de 15.2 y la pobreza en Texas (el estado donde se
encuentra Houston) es de 11.1, la diferencia es 4.1.(15 pts)

b) Obtener la cantidad de muertes de gente de raza negra por parte
de oficiales de policía, agrupada por estados que compartan el mismo
nivel de pobreza redondeado al entero más cercano. Por ejemplo, si
NJ tiene una pobreza de 10.33, AL una de 20.64 y AZ una de 10.44,
NJ y AZ quedarían juntos representados por el nivel de pobreza de 10
y AL en otro grupo con el nivel 21. La salida debe tener el formato:
(nivel_de_pobreza, total_de_muertes) (15 pts)

Resolver los puntos usando la API de RDDs de PySpark. (30 pts)

In [49]:
import pyspark as ps
import pandas as pd

# create the Spark Session
spark = ps.sql.SparkSession.builder.getOrCreate()
# create the Spark Context
sc = spark.sparkContext

murdersRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2020-1C-1Op.csv', header=True).rdd
povertyRdd = spark.read.csv('/home/marcos/PycharmProjects/PracticaSpark/Parciales/P2020-1C-2Op.csv', header=True).rdd.cache()

povertyPerState = povertyRdd.map(lambda x: (x[0], (pd.to_numeric(x[2], 'coerce'), 1))) \
            .reduceByKey(lambda x,y: (x[0] + y[0], x[1] + y[1])).mapValues(lambda x: x[0]/x[1]).cache()

povertyRdd.map(lambda x: (x[0], (x[1], pd.to_numeric(x[2], 'coerce')))).join(povertyPerState) \
            .takeOrdered(10, key=lambda x: -(x[1][0][1] - x[1][1]))

[('S3', (('Washington DC', 0.95), 0.4483333333333333)),
 ('S3', (('San Francisco', 0.89), 0.4483333333333333)),
 ('S2', (('New York', 0.65), 0.36666666666666664)),
 ('S1', (('San Francisco', 0.7), 0.44833333333333325)),
 ('S2', (('Los Angeles', 0.56), 0.36666666666666664)),
 ('S1', (('Chicago', 0.6), 0.44833333333333325)),
 ('S2', (('Chicago', 0.47), 0.36666666666666664)),
 ('S1', (('New York', 0.55), 0.44833333333333325)),
 ('S2', (('New Jersey', 0.32), 0.36666666666666664)),
 ('S1', (('Washington DC', 0.4), 0.44833333333333325))]

In [50]:
povertyPerState = povertyPerState.mapValues(lambda x: int(round(x*100)))

murdersRdd.map(lambda x: (x[4], (1 if (x[2] == 'Black') else 0))).reduceByKey(lambda x,y: x + y) \
         .join(povertyPerState).map(lambda x: (x[1][1], x[1][0])) \
        .reduceByKey(lambda x,y: x + y).collect()


[(45, 125), (37, 68)]